# TRAIN DATA
---
---
### *Using code from code notebooks 00_1_garmin_gpx.ipynb & 00_2_openweather_api.ipynb to build out the two training datasets for female average performance and high performance. See code notebooks for sources on code.*

# Imports

In [1]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import time

# GPX file
import gpxpy
import gpxpy.gpx

# GPX file
from bs4 import BeautifulSoup
import lxml

# bearing
from geographiclib.geodesic import Geodesic

# distance
import haversine as hs

# API
import requests

%matplotlib inline
pd.set_option('display.max_columns', None)

import warnings
warnings.simplefilter("ignore")

In [2]:
API_key = [REDACTED]

# Functions
---

## Historical Weather

In [3]:
def hist_weather(unix_datetime, lat, lon, API_key = API_key):
    '''
    Using the OpenWeather OneCall 3.0 API to pull historical daily data
    Requires API key for at minimum Startup OpenWeather subscription
    Input: UNIX datetime for day requesting
    Output: Pandas dataframe for that historical datetime's weather
    '''
    
    url_hist_point = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={unix_datetime}&appid={API_key}"
    hist_point_req = requests.get(url_hist_point)

    wd_hist = hist_point_req.json()

    wd_hist_df = json_normalize(wd_hist['data'])
    wd_hist_df = wd_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
           'dew_point', 'clouds', 'wind_speed', 'wind_deg']]

    
    return wd_hist_df

## GPX to Dataframe

In [4]:
def gpx_to_df(filepath):
    
    # Open .gpx file and parse xml
    with open(filepath, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
        
    # check file
    print(f'File: {filepath}')
    # check length of tracks
    print(f'Tracks: {len(gpx.tracks)}')
    # check number of data points
    print(f'No. of Track Points: {gpx.get_track_points_no()}')
    
    # extract timestamp, lat, lon, and elevation from gpx file
    route_info = []

    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                route_info.append({
                    'timestamp': point.time,
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'elevation': point.elevation,
                })
    
    # create dataframe
    route_df = pd.DataFrame(route_info)
    # create UNIX column for later
    # https://statisticsglobe.com/convert-datetime-to-unix-timestamp-python
    route_df['dt'] = route_df['timestamp'].apply(lambda z: int(datetime.datetime.timestamp(z)))
    
    #######################
    # HEART RATE - REMOVED CADENCE DUE TO LACK OF SENSOR DATA EARLIER THIS YEAR
    # add extension data with beautifulsoup
    xml = gpx.to_xml()
    soup = BeautifulSoup(xml, features = 'xml')
    
    # Use Beautiful Soup to find all heart rate
    hr_soup = soup.find_all('ns3:hr')

    # for loop to extract heart rate and cadence values from each point
    hr_list = []
    for i in hr_soup:
        # https://stackoverflow.com/questions/69420686/bs4-element-resultset-elements-to-a-list
        hr_list.append(i.get_text(strip = True))

    # add columns to dataset from extraction
    route_df['heart_rate'] = hr_list
    
    #######################
    # BEARING
    # get_bearing function
    def get_bearing(lat1, lat2, long1, long2):
        brng = Geodesic.WGS84.Inverse(lat1, long1, lat2, long2)['azi1']
        return brng
    
    # assign bearing in df
    route_df['bearing'] = 0
    for i in range(1, len(route_df)):
        lat1 = route_df.latitude.iloc[i-1]
        lat2 = route_df.latitude.iloc[i]
        long1 = route_df.longitude.iloc[i-1]
        long2 = route_df.longitude.iloc[i]
        bearing = get_bearing(lat1, lat2, long1, long2)
        # https://stats.stackexchange.com/questions/283572/using-iloc-to-set-values
        route_df.bearing.iloc[[i]] = bearing
    
    ########################    
    # ELAPSED TIME - SECONDS
    route_df['timestamp'] = pd.to_datetime(route_df['timestamp'])
    route_df['time_diff_s'] = 0
    for i in range(1, len(route_df)-1):
        t1 = route_df.iloc[i]['timestamp']
        t2 = route_df.iloc[i-1]['timestamp']
        # https://www.geeksforgeeks.org/how-to-set-cell-value-in-pandas-dataframe/
        route_df.at[i, 'time_diff_s'] = (t1 - t2).seconds
    route_df['total_time_s'] = route_df['time_diff_s'].cumsum(skipna = True)
    
    #######################
    # ELEVATION CHANGE - METERS
    route_df['ele_diff_m'] = 0
    for i in range(1, len(route_df)-1):
        e1 = route_df.iloc[i]['elevation']
        e2 = route_df.iloc[i-1]['elevation']
        route_df.at[i, 'ele_diff_m'] = (e1 - e2)
    route_df['total_ele_change_m'] = round(route_df['ele_diff_m'].cumsum(skipna = True), 4)
    
    #######################
    # DISTANCE - KILOMETERS
    route_df['lat_lon'] = [(lat, lon) for lat, lon in zip(route_df['latitude'], route_df['longitude'])]
    route_df['dist_diff_km'] = 0
    for i in range(1, len(route_df)-1):
        loc1 = route_df.iloc[i]['lat_lon']
        loc2 = route_df.iloc[i-1]['lat_lon']
        route_df.at[i, 'dist_diff_km'] = round(hs.haversine(loc1, loc2), 4) # kilometers
    route_df['total_dist_km'] = route_df['dist_diff_km'].cumsum(skipna = True)
    
   
    #######################
    route_df.reset_index(drop = True, inplace = True)
    
    return route_df

# Average Cycling Performance Period Dataset

## Get GPX files and create dataframe

In [5]:
'''
#https://stackoverflow.com/questions/13603215/using-a-loop-in-python-to-name-variables
average = dict()
 
for x in range(1, 13): # number of files
    average[x] = gpx_to_df(f'../data/average/a_{x}.gpx')

a_df = pd.concat(average)
a_df.dropna(inplace = True)
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html
a_df.reset_index(inplace = True, level = [0,1], drop = True)

print()
print(f'Size of a_df: {a_df.shape}')
print()

a_df.head()
'''

"\n#https://stackoverflow.com/questions/13603215/using-a-loop-in-python-to-name-variables\naverage = dict()\n \nfor x in range(1, 13): # number of files\n    average[x] = gpx_to_df(f'../data/average/a_{x}.gpx')\n\na_df = pd.concat(average)\na_df.dropna(inplace = True)\n# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html\na_df.reset_index(inplace = True, level = [0,1], drop = True)\n\nprint()\nprint(f'Size of a_df: {a_df.shape}')\nprint()\n\na_df.head()\n"

## Get historical weather and join with average dataframe

In [6]:
hist_weather(1658333265, 38.752125, -121.288010)

,dt,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg
0,1658333265,298,297.53,1017,38,282.73,1,0.45,177


In [7]:
'''
a_weather = pd.DataFrame()

for i in range(len(a_df)): # iterate through each trackpoint
    # 600 calls per minute
    unix = a_df['dt'][i]
    lat = a_df['latitude'][i]
    lon = a_df['longitude'][i]
    weather = hist_weather(unix, lat, lon) # get historical weather for trackpoint
    a_weather = a_weather.append(weather) # append to weather dataframe
        
a_df = a_df.merge(a_weather, how = 'inner', on = 'dt') # merge activity dataframe with weather dataframe on unix time stamp
'''

"\na_weather = pd.DataFrame()\n\nfor i in range(len(a_df)): # iterate through each trackpoint\n    # 600 calls per minute\n    unix = a_df['dt'][i]\n    lat = a_df['latitude'][i]\n    lon = a_df['longitude'][i]\n    weather = hist_weather(unix, lat, lon) # get historical weather for trackpoint\n    a_weather = a_weather.append(weather) # append to weather dataframe\n        \na_df = a_df.merge(a_weather, how = 'inner', on = 'dt') # merge activity dataframe with weather dataframe on unix time stamp\n"

In [8]:
# a_df

# High Cycling Performance Period Dataset

## Get GPX files and create dataframe

In [9]:
'''
#https://stackoverflow.com/questions/13603215/using-a-loop-in-python-to-name-variables
high = dict()

for x in range(1, 17): # number of files
    high[x] = gpx_to_df(f'../data/high/h_{x}.gpx')

h_df = pd.concat(high)
h_df.dropna(inplace = True)
h_df.reset_index(inplace = True, level = [0,1], drop = True)
print()
print(f'Size of h_df: {h_df.shape}')
print()
h_df.head()
'''

"\n#https://stackoverflow.com/questions/13603215/using-a-loop-in-python-to-name-variables\nhigh = dict()\n\nfor x in range(1, 17): # number of files\n    high[x] = gpx_to_df(f'../data/high/h_{x}.gpx')\n\nh_df = pd.concat(high)\nh_df.dropna(inplace = True)\nh_df.reset_index(inplace = True, level = [0,1], drop = True)\nprint()\nprint(f'Size of h_df: {h_df.shape}')\nprint()\nh_df.head()\n"

## Get historical weather and join with high dataframe

In [10]:
'''
h_weather = pd.DataFrame()

for i in range(len(h_df)): # iterate through each trackpoint
    # 600 calls per minute
    unix = h_df['dt'][i]
    lat = h_df['latitude'][i]
    lon = h_df['longitude'][i]
    weather = hist_weather(unix, lat, lon) # get historical weather for trackpoint
    h_weather = h_weather.append(weather) # append to weather dataframe
        
h_df = h_df.merge(h_weather, how = 'inner', on = 'dt') # merge activity dataframe with weather dataframe on unix time stamp
'''

"\nh_weather = pd.DataFrame()\n\nfor i in range(len(h_df)): # iterate through each trackpoint\n    # 600 calls per minute\n    unix = h_df['dt'][i]\n    lat = h_df['latitude'][i]\n    lon = h_df['longitude'][i]\n    weather = hist_weather(unix, lat, lon) # get historical weather for trackpoint\n    h_weather = h_weather.append(weather) # append to weather dataframe\n        \nh_df = h_df.merge(h_weather, how = 'inner', on = 'dt') # merge activity dataframe with weather dataframe on unix time stamp\n"

In [11]:
# h_df.head()

# Save CSV

In [12]:
# a_df.to_csv('../data/average/a_df.csv', index = False)
# h_df.to_csv('../data/high/h_df.csv', index = False)